In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [60]:
#Import Bibliotheques
import pandas as pd
import numpy as np

In [61]:
#Importation des fichiers csv 
folder_path = "/kaggle/input/melting-point"
train_csv_path = '/kaggle/input/melting-point/train.csv'
train_csv = pd.read_csv(train_csv_path)
#print(train_csv.head())

test_csv_path = '/kaggle/input/melting-point/test.csv'
test_csv = pd.read_csv(test_csv_path)
#print(test_csv.head())

In [62]:
#Chargement d'un Dataset léger pour tester mon code
#Low weight dataset loding - in order to test my code before the big loading
#train_test_csv = train_csv.head(1000)
#train_data_set = train_test_csv

#Chargement du Dataset complet
#Full dataset loading
train_data_set = train_csv

y = train_data_set['Tm']
list_columns = train_data_set.columns.tolist()
feature_names = list_columns[3:]

# print the list of columns in the dataset to find the name of the prediction target
#train_data_set.describe()

test_data_set = test_csv

In [63]:
#Retraitement des données sous la forme de 0 et 1 - Train
X_df = pd.DataFrame(train_data_set[feature_names])

#Créer les colonnes binaires
binary_feats = {f"has_{col}": (X_df[col] > 0).astype(int) for col in feature_names}

#Ratios 
ratio_feats = {f"{col}_ratio": X_df[col] / (X_df[feature_names].sum(axis=1) + 1e-6) for col in feature_names}

#Totaux globaux
extra_feats = {
    "total_groups": X_df[feature_names].sum(axis=1),
    "nb_groups_non_zero": (X_df[feature_names] > 0).sum(axis=1),
}

# 5. Fusionner toutes les nouvelles colonnes d’un coup
X_df = pd.concat([X_df, pd.DataFrame({**binary_feats, **ratio_feats, **extra_feats})], axis=1)

print("Shape après feature engineering:", X_df.shape)

Shape après feature engineering: (2662, 1274)


In [64]:
#Retraitement du fichier Test
X_test_df = pd.DataFrame(test_data_set[feature_names])
#Créer les colonnes binaires
binary_feats_test = {f"has_{col}": (X_test_df[col] > 0).astype(int) for col in feature_names}
ratio_feats_test = {f"{col}_ratio": X_test_df[col] / (X_test_df[feature_names].sum(axis=1) + 1e-6) for col in feature_names}
extra_feats_test = {
    "total_groups": X_test_df[feature_names].sum(axis=1),
    "nb_groups_non_zero": (X_test_df[feature_names] > 0).sum(axis=1),
}
X_test_df = pd.concat([X_test_df, pd.DataFrame({**binary_feats_test, **ratio_feats_test, **extra_feats_test})], axis=1)
print("Shape après feature engineering:", X_test_df.shape)

Shape après feature engineering: (666, 1274)


In [65]:
X = X_df
#print(X.head())
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Split Train / Valid
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.2, random_state=42 )


**Import et Selection du modèle**

In [66]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [57]:
final_model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.09586251212066181,
    depth=10,
    l2_leaf_reg=3,
    bagging_temperature=2.031454661334689,
    random_state=42,
    verbose=100
)
final_model.fit(X, y)

0:	learn: 83.7684163	total: 19.6ms	remaining: 39.2s
100:	learn: 55.7672322	total: 1.25s	remaining: 23.5s
200:	learn: 49.1295828	total: 2.49s	remaining: 22.3s
300:	learn: 43.8493470	total: 3.71s	remaining: 21s
400:	learn: 40.5721797	total: 4.93s	remaining: 19.7s
500:	learn: 37.8775281	total: 6.24s	remaining: 18.7s
600:	learn: 35.7731561	total: 7.46s	remaining: 17.4s
700:	learn: 34.1702845	total: 8.7s	remaining: 16.1s
800:	learn: 32.8274436	total: 9.92s	remaining: 14.9s
900:	learn: 31.6214961	total: 11.1s	remaining: 13.6s
1000:	learn: 30.5964848	total: 12.4s	remaining: 12.4s
1100:	learn: 29.7976656	total: 13.6s	remaining: 11.1s
1200:	learn: 28.8901791	total: 14.9s	remaining: 9.9s
1300:	learn: 28.0767742	total: 16.2s	remaining: 8.68s
1400:	learn: 27.3350537	total: 17.4s	remaining: 7.43s
1500:	learn: 26.7308904	total: 18.6s	remaining: 6.18s
1600:	learn: 26.0913519	total: 19.9s	remaining: 4.95s
1700:	learn: 25.5332149	total: 21.1s	remaining: 3.71s
1800:	learn: 24.9394761	total: 22.3s	remain

In [58]:
#Test de modèle sur données de test
X_test = X_test_df
predictions = final_model.predict(X_test)
print(predictions)

[334.85162129 304.18649363 200.84100409 193.81907118 272.59678911
 338.3566191  229.39272162 302.58060096 264.93725713 271.68097245
 282.5866567  307.36325975 300.45196423 260.55068828 253.47255248
 604.09361045 266.52243791 263.9360948  371.03079141 164.45915156
 154.96755795 166.7385768  395.69600748 213.00280192 288.50383348
 188.20815837 615.48808347 517.69847783 394.13828044 298.12449554
 371.94824644 274.26823442 288.40673283 197.67851271 317.83658361
 262.81450404 415.71557766 229.29701311 259.04635973 337.80133038
 205.25167085 265.01483727 268.40683714 160.10809491 169.71012259
 309.98289603 323.25383794 252.36044813 298.46633298 332.80760053
 317.37351308 323.63053142 127.65016288 296.93554728 286.30107853
 286.89038571 238.8650699  252.28033568 266.3714547  243.095077
 324.88876245 318.30930451 196.15786509 310.24082506 346.43894676
 205.88153604 267.74734663 380.99178861 271.92472784 306.8238534
 290.28692164 372.86877114 199.72494312 205.90566983 159.4203373
 277.02559638 

In [44]:
#Export au format csv
id = test_data_set['id']
Tm = predictions
dict = {"id": id, "Tm": Tm}
submission_df = pd.DataFrame(dict)

# Sauvegarder le DataFrame en fichier CSV
output_csv_path = "submission.csv"
submission_df.to_csv(output_csv_path, index=False, sep=',')

print(f"Fichier CSV généré : {output_csv_path}")
print(submission_df)

Fichier CSV généré : submission.csv
       id          Tm
0    1022  339.148700
1    1146  299.590042
2      79  211.319675
3    2279  200.832867
4    1342  257.140557
..    ...         ...
661  2663  266.841786
662   624  288.457148
663  2655  174.489529
664  2089  239.724927
665  1065  276.841424

[666 rows x 2 columns]


In [ ]:
from IPython.display import FileLink
FileLink("submission.csv")